<a href="https://colab.research.google.com/github/Vaelastraszz/Stack_OF/blob/master/Stack_Overflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
raw_post = pd.read_csv("/content/gdrive/My Drive/Stack_OF/QueryResults2.csv")

In [0]:
pd.options.display.max_columns = None

In [5]:
raw_post.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,4,1,7.0,NaN,2008-07-31 21:42:52,NaN,605,39296.0,<p>I want to use a track-bar to change a form'...,8.0,NaN,6786713.0,Rich B,2018-07-02 17:55:27,2019-01-17 13:39:48,Convert Decimal to Double?,<c#><floating-point><type-conversion><double><...,13.0,2,46.0,NaN,2012-10-31 16:42:47
1,6,1,31.0,NaN,2008-07-31 22:08:08,NaN,274,17393.0,<p>I have an absolutely positioned <code>div</...,9.0,NaN,63550.0,Rich B,2016-03-19 06:05:48,2018-12-15 03:57:18,Percentage width child element in absolutely p...,<html><css><css3><internet-explorer-7>,6.0,0,11.0,NaN,NaN
2,7,2,NaN,4.0,2008-07-31 22:17:57,NaN,419,NaN,<p>An explicit cast to double like this isn't ...,9.0,NaN,4020527.0,NaN,2017-12-16 05:06:57,2017-12-16 05:06:57,NaN,NaN,NaN,0,NaN,NaN,NaN
3,9,1,1404.0,NaN,2008-07-31 23:40:59,NaN,1743,522602.0,<p>Given a <code>DateTime</code> representing ...,1.0,NaN,3956566.0,Rich B,2018-04-21 17:48:14,2018-07-25 11:57:14,How do I calculate someone's age in C#?,<c#><.net><datetime>,63.0,7,421.0,NaN,2011-08-16 19:40:43
4,11,1,1248.0,NaN,2008-07-31 23:55:37,NaN,1415,143330.0,<p>Given a specific <code>DateTime</code> valu...,1.0,NaN,6479704.0,user2370523,2017-06-04 15:51:19,2019-01-03 08:03:07,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,36.0,3,536.0,NaN,2009-09-04 13:15:59


In [6]:
raw_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40161 entries, 0 to 40160
Data columns (total 22 columns):
Id                       40161 non-null int64
PostTypeId               40161 non-null int64
AcceptedAnswerId         6576 non-null float64
ParentId                 32180 non-null float64
CreationDate             40161 non-null object
DeletionDate             0 non-null float64
Score                    40161 non-null int64
ViewCount                7981 non-null float64
Body                     40161 non-null object
OwnerUserId              38932 non-null float64
OwnerDisplayName         34788 non-null object
LastEditorUserId         15670 non-null float64
LastEditorDisplayName    9029 non-null object
LastEditDate             15860 non-null object
LastActivityDate         40161 non-null object
Title                    7981 non-null object
Tags                     7981 non-null object
AnswerCount              7981 non-null float64
CommentCount             40161 non-null int64
Favor

<h1> Nettoyage des données </h1>

In [0]:
post = raw_post[["Body", "Title", "Tags"]]

Réduction du dataframe avec les seulement les colonnes qui reflètent un suceptible tagging

In [108]:
duplicates = post[post.Body.map(post.Body.value_counts() > 1)]
duplicates.sort_values('Body')[['Body', 'Title']]

,Body,Title
17113,<p>Anyone have any idea? And any open source s...,How to check the overall health of database us...
17114,<p>Anyone have any idea? And any open source s...,Monitor SQL currently in shared pool using Toad
17115,<p>Anyone have any idea? And any open source s...,Monitoring a database instance
17118,<p>Anyone have any idea? And any open source s...,How to use SQLab Xpert Tuning to tune SQL for ...
10253,<p>Could somebody please name a few. I could ...,Advantages of VS 2008 over VS 2005
10254,<p>Could somebody please name a few. I could ...,Benefits of SQL Server 2005 over 2000
30089,<p>How can I turn off the wavy red underlines ...,Netbeans - Turn off - wavy red underlines - How?
30186,<p>How can I turn off the wavy red underlines ...,Netbeans 6.1 (!5.5.1) - Turn off - wavy red un...
31152,<p>I am attempting to deploy an ASP.NET 2.0 We...,Need help configuring IIS to serve 2.0 web ser...
31155,<p>I am attempting to deploy an ASP.NET 2.0 We...,Need help configuring IIS to serve 2.0 web ser...


Observation des doublons sur le titre et le body puis suppression de ces doublons

In [109]:
 post.drop_duplicates(['Body', 'Title'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [110]:
post.head()

,Body,Title,Tags
0,<p>I want to use a track-bar to change a form'...,Convert Decimal to Double?,<c#><floating-point><type-conversion><double><...
1,<p>I have an absolutely positioned <code>div</...,Percentage width child element in absolutely p...,<html><css><css3><internet-explorer-7>
2,<p>Given a <code>DateTime</code> representing ...,How do I calculate someone's age in C#?,<c#><.net><datetime>
3,<p>Given a specific <code>DateTime</code> valu...,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...
4,<p>Is there a standard way for a web server to...,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...


In [111]:
post.isnull().sum().sort_values(ascending=False) / post.shape[0] * 100

Tags     0.0
Title    0.0
Body     0.0
dtype: float64

Observation des valeurs nulles dans le dataframe afin d'être sur d'avoir un echantillon de données complet pour la suite 

In [112]:
post["Full_Post"] = np.where(post["Title"].isna(),post["Body"] , post["Title"] + " " + post["Body"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Concatenation du body et du titre pour ne perdre aucune information sur le suceptible post (I.E un langage peut être juste mentionné dans le titre puis suggeré dans le post ainsi on perdrait de l'information sur le post et le système de tagging)

In [0]:
post = post["Full_Post"]

<h2> Preprocessing des données avec NLTK + BeautifulSoup </h2>

<h3> BeautifulSoup </h3>

In [0]:
import nltk
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

In [0]:
def clean_code(post):
    for i in range(len(post)):
        post[i]=str(post[i]).replace(str(BeautifulSoup(post[i]).find("code")), "")
        post[i]= BeautifulSoup(post[i]).get_text()

Utilisation de la libraire BeautifuSoup afin de d'abord supprimer le code dans les posts puis ensuite récupérer le texte qui se trouve entre les tags HTML 

In [0]:
liste_p = post.tolist()

Transformation de la série en liste afin de faciliter les futurs pre processing plutot qu'utiliser une serie qui ralentirait beaucoup les traitements.

In [0]:
clean_code(liste_p)

<h3> NLTK Processing </h3>

In [0]:
import pickle

liste_p = pickle.load(open('/content/gdrive/My Drive/Stack_OF/liste_posts.pkl','rb'))

In [0]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['need','think','way','want','problem','think','question',
                   'answer','use','ii','change','help','would',"write",'check','sure','error','exists','work','thing','like','look',
                  "something", "could","simple","might" ,"along", "though","guess", "course" ,"anything","one","another","two","lot",
                  "simple","language","code","garbage","either","wait","foo","idea","org","solution","found"])

Création d'un ensemble de stop words afin de pouvoir supprimer tous les mots vide de sens pour les futur topics générer par notre algorithme non supervisé

In [0]:
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize with POS Tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

Création d'une fonction permettant de renvoyer le type de chaque mot (adjectif, verbe, noms, adverbes ). L'objectif est de garder uniquement les noms car notre algorithme de tagging doit suceptiblement faire apparaitre des langages informatiques, destechnologies ou d'autre chose liées à Stack Overflow directement ce q'un verbe ne ferait pas et diminuerait la pertinence des taggings.

In [0]:
def tokenize_list(liste_p):

    new_list = list()
    freq =  nltk.FreqDist()
    
    for i in range(len(liste_p)):
        
        new_list.append([lemmatizer.lemmatize(w.lower(),get_wordnet_pos(w.lower())) 
                                      for w in tokenizer.tokenize(liste_p[i]) if not w.lower() in stop_words and str.isalpha(w)== True and
                         get_wordnet_pos(w.lower())=="n"])
        
        freq += nltk.FreqDist(new_list[i])
        
    return new_list, freq

Création d'une fonction renvoyant une liste de fréquence pour observer les mots qui ressortent le plus pour peut être updater notre liste de stop words si on observer des mots incohérents. On renvoie aussi une liste traiter avec lemmatization + suppression des variable numériques + recuperation du bag of words en minuscule

In [0]:
new_list,freq = tokenize_list(liste_p)

In [128]:
freq.most_common(100)

[('file', 18710),
 ('c', 14056),
 ('application', 12135),
 ('net', 11802),
 ('server', 11007),
 ('user', 10410),
 ('data', 10000),
 ('class', 9726),
 ('window', 8858),
 ('object', 8434),
 ('web', 8060),
 ('time', 7897),
 ('page', 7751),
 ('value', 7560),
 ('method', 7465),
 ('project', 7365),
 ('database', 7229),
 ('string', 7132),
 ('table', 6883),
 ('sql', 6812),
 ('system', 6785),
 ('example', 6561),
 ('set', 6524),
 ('control', 6460),
 ('java', 6357),
 ('function', 6335),
 ('type', 6309),
 ('name', 5866),
 ('anyone', 5622),
 ('test', 5325),
 ('create', 5189),
 ('list', 5166),
 ('asp', 5102),
 ('text', 4927),
 ('service', 4774),
 ('form', 4677),
 ('line', 4631),
 ('call', 4595),
 ('return', 4430),
 ('version', 4355),
 ('number', 4261),
 ('app', 4223),
 ('library', 4109),
 ('access', 4082),
 ('http', 4077),
 ('thanks', 4070),
 ('edit', 4031),
 ('xml', 3972),
 ('update', 3956),
 ('without', 3916),
 ('source', 3847),
 ('property', 3700),
 ('event', 3675),
 ('client', 3671),
 ('php', 36

Mots en top fréquence (100 plus fréquents)

<h3> Tf-idf </h3>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

In [0]:
from nltk.tokenize.moses import MosesDetokenizer

detokenizer = MosesDetokenizer()

new_list = [detokenizer.detokenize(x, return_str=True) for x in new_list]

Detokenization des  bags of words afin de pouvoir appliquer directement la transformation tf_idf sans avoir a utiliser de boucle dans nos sous liste 

In [0]:
new_list = [x.replace("cs","css") for x in new_list]

Correction sur le mot css au lieu de cs qui a été tronqué par la lemmatization 

In [0]:
vectorizer =  TfidfVectorizer(min_df=2, max_features=4000, stop_words= stop_words)

tf_idf = vectorizer.fit_transform(new_list)

Application de la transformation en TF_Idf en une matrice (nb_posts, mots) avec chacun leur poids tf_idf 

In [0]:
vectorizer.get_feature_names()

Récupération du nom des features de la matrice tf_idf

<h3> Test Non negative matrix factorisation </h3>

In [0]:
from sklearn.decomposition import NMF

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

Création d'une fonction pour afficher les 10 mots les plus impactant au sein de chaque topics

In [126]:
no_topics = 35

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tf_idf)

no_top_words = 10
display_topics(nmf, vectorizer.get_feature_names(), no_top_words)

Topic 0:
sql server query procedure linq client connection express statement management
Topic 1:
table column row query id select record sql mysql key
Topic 2:
file directory folder path read copy config upload script batch
Topic 3:
class public interface name member instance base property constructor type
Topic 4:
page javascript html browser site web css content link ajax
Topic 5:
net asp mvc framework vb web website ajax site webforms
Topic 6:
string character convert regex expression char match number format return
Topic 7:
studio debug express ide microsoft build option management designer debugger
Topic 8:
project build source eclipse folder svn reference repository dependency directory
Topic 9:
service web wcf client soap call server wsdl config site
Topic 10:
java eclipse library jar jvm applet lang sun api equivalent
Topic 11:
function call return javascript parameter argument pointer int jquery pas
Topic 12:
test unit nunit junit framework coverage integration mock tdd suite


Création de la matrice non négative que l'on va fit avec notre matrice TF_idf.
La matrice TF_idf va être décomposée en une matrice V = WH


*   V notre matrice initiale tf_idf (n_post,n_mots)
*   W notre matrice de posts to topics (n_post, n_topics)
*   H notre components matrix donc topics to words (n_topics, n_words)





<h3> Export tf_idf Matrix + Liste posts</h3>

In [0]:
import pickle

pickle.dump(tf_idf, open('/content/gdrive/My Drive/Stack_OF/tf_idf.pkl','wb'))
pickle.dump(liste_p, open('/content/gdrive/My Drive/Stack_OF/liste_posts.pkl','wb'))
pickle.dump(vectorizer.get_feature_names(), open('/content/gdrive/My Drive/Stack_OF/terms.pkl','wb'))
pickle.dump(vectorizer, open('/content/gdrive/My Drive/Stack_OF/vectorizer.pkl','wb'))

Sérialisation des instances qui seront utiles dans nos autres notebook

<h2> Download packages NLTK </h2>

In [5]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('perluniprops')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.


True

Téléchargement de tous les packages utiles a nos preprocessing avec NLTK